**Importy**

In [35]:
import regex
from csv import writer
import pandas as pd 
import numpy as np
from stop_words import get_stop_words

In [36]:
import majka
from majka import Majka
morph = Majka('../data/w-lt.sk.fsa')

morph.flags |= majka.ADD_DIACRITICS  # find word forms with diacritics
morph.flags |= majka.DISALLOW_LOWERCASE  # do not enable to find lowercase variants
morph.flags |= majka.IGNORE_CASE  # ignore the word case whatsoever
morph.flags = 0  # unset all flags

morph.tags = False  # return just the lemma, do not process the tags
# morph.tags = True  # turn tag processing back on (default)

morph.first_only = True  # return only the first entry
# morph.first_only = False  # return all entries (default)

In [37]:
from corpy.morphodita import Tagger
tagger = Tagger("../data/slovak-morfflex-pdt-170914.tagger")

INFO:corpy.morphodita:Loading tagger.


In [38]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Premenne**

In [60]:
debug_filepath = "../data/test_data_1page.xml"
input_filepath = "../data/skwiki-latest-pages-articles.xml"
output_filepath = "../data/anchors.csv"

**Funkcie**

In [51]:
def open_read(input_filepath,output_filepath,process):
    with open(input_filepath, encoding='UTF-8') as input_file ,open(output_filepath, 'w+', encoding='UTF-8' ) as output_file:
        csv_writer = writer(output_file, delimiter='|')
        csv_writer.writerow(["Link", "Alt_Text"])
        for line in input_file:
            process(line,csv_writer)

In [52]:
def process(line,csv_writer):
    regex_patern = '\[\[([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)\|?([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)?\]\]'
    matches = regex.findall(regex_patern, line)
    for match in matches:
        list_matche = list(filter(None, match))
        
#         print(list_matche)
        
        csv_writer.writerow(list_matche)

In [53]:
def replace(text, regex_pattern, substitution):
    if not isinstance(text, str):
        return text
    return regex.sub(regex_pattern,substitution,text)

In [54]:
def tokenizer_modded(text):
    sentences = nltk.sent_tokenize(text)
    tokens = []
    for sent in sentences:
        tokens.append(nltk.word_tokenize(sent))
    flatten_list = [j for sub in tokens for j in sub] 
    result = [token.lower() for token in flatten_list if token not in ".,?!...-"]
    
    return result

In [55]:
def tag(word):
    result=[]
    tokens = list(tagger.tag(word))
#     for i in range(len(tokens)):
#         if any(s in tokens[i].tag for s in need):
#             result.append(tokens[i].word)
            
    return tokens

In [96]:
def lematizer_modded(text):
    stop_words = get_stop_words('sk')
#     stop_words2 = ['a','s','v',' s',' a',' v','z','zo','o','u','1','2','3','4','5','6','7','8','9','0','–']
#     stop_words = stop_words + stop_words2
    result = []
    for word in text:
        morph_word = morph.find(word)
        if morph_word:
            result.append(morph_word[0]['lemma'])
        else:
            result.append(word)
    for word in result:  # iterating on a copy since removing will mess things up
        if word in stop_words:
            result.remove(word)
    return result

**Priebeh spracovania**

In [97]:
open_read(debug_filepath,output_filepath,process)

In [100]:
for index,chunk in enumerate(pd.read_csv(output_filepath, chunksize=15, delimiter='|')):
    chunk['Link'] = chunk['Link'] \
    .apply(lambda x: x if x is not np.NaN else None) \
    .apply(replace,regex_pattern = r"[\\\/\(\):#.\_]|({{.*}})|\d*px\png\svg\jpg",substitution =' ')\
    .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
    .map(lambda text: lematizer_modded(tokenizer_modded(text)) if text is not None else None)
    
    chunk['Alt_Text'] = chunk['Alt_Text'] \
    .apply(lambda x: x if x is not np.NaN else None) \
    .apply(replace,regex_pattern = r"[\\\/\(\):#.\_]|({{.*}})|\d*px\png\svg\jpg",substitution =' ')\
    .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
    .map(lambda text: lematizer_modded(tokenizer_modded(text)) if text is not None else None )
    
    
    print('chunk ------------------------------------------------------------------------------',index)    
    print(chunk)
    chunk.to_csv('../data/done.csv', mode = 'a+', index=False, encoding='UTF-8')


chunk ------------------------------------------------------------------------------ 0
                                                 Link  \
0                                         [slovensko]   
1               [slovenský, republika, 1939, –, 1945]   
2                                         [slovensko]   
3                                      [generálmajor]   
4                                  [ľubomír, svoboda]   
5   [dopravné, krídlo, generál, milana, rastislava...   
6   [taktický, krídlo, generálmajor, otta, smika, ...   
7   [vrtuľníkový, krídlo, generálplukovník, jána, ...   
8         [protilietadlový, raketový, brigáda, nitra]   
9      [brigáda, riadenie, vzdušný, operácia, zvolen]   
10              [súbor, official-emblem-of-ossr, png]   
11            [ozbrojený, sila, slovenský, republika]   
12                 [súbor, coat, arms, slovakia, svg]   
13  [súbor, roundel, slovakia, –, low, visibility,...   
14                             [aero, l-39, albatros]   



chunk ------------------------------------------------------------------------------ 10
                                                  Link  \
150                     [beechcraft, super, king, air]   
151                                          [28, júl]   
152                                             [2015]   
153                                       [mil, mi-17]   
154                                            [terňa]   
155                                     [podpráporčík]   
156                         [armáda, český, republika]   
157                                      [15, február]   
158                                             [2017]   
159                               [aero, l-29, delfín]   
160  [nato, integrated, air, missile, defence, system]   
161                                            [sliač]   
162                          [kováčová, okres, zvolen]   
163                       [stredoeurópsky, letný, čas]   
164                             [aero, l-3

chunk ------------------------------------------------------------------------------ 26
                                                  Link            Alt_Text
390  [súbor, 122ai, slovak, republic, tupolev, 154m...             [150px]
391                                  [sovietsky, zväz]              [zssr]
392                                 [cvičný, lietadlo]  [cvičný, lietadlo]
393                               [aero, l-29, delfín]                None
394           [súbor, l-29, sovakia, 22211294871, jpg]             [150px]
395                                  [česko-slovensko]                None
396                                      [helikoptéra]          [vrtuľník]
397                                        [mil, mi-2]                None
398          [súbor, mi-2, slovakia, 22589902916, jpg]             [150px]
399                                  [sovietsky, zväz]              [zssr]
400                                           [poľsko]                None
401         